In [1]:
#ライブラリ読み込み
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium.common.exceptions import NoSuchElementException

In [2]:
#スクレイピングリスト読み込み
DATA_IN_PATH="C:/r_analysis/1.20221101/"
kaisha=pd.read_csv(DATA_IN_PATH + 'newlist.csv',header=0,encoding="cp932",quoting=3)
h=kaisha['file_list'][0:30]
l=kaisha['name'][0:30]

In [ ]:
#chrome driverでサイトに接続
url = 'https://en-hyouban.com/'
driver = webdriver.Chrome('/r_analysis/chromedriver.exe') 
driver.get(url)
driver.implicitly_wait(10)

#log-in
search_box = driver.find_element('css selector', "#content-banner > div.navbar-mobile.row.d-lg-none.d-xl-none.d-xs-block > div:nth-child(1) > div.col-lg-6.text-white.col-md-2.col-sm-2.w-auto.ml-auto > div.text-right.align-middle > a > em")
search_box.click()
driver.implicitly_wait(5)
search_box = driver.find_element('css selector', "#company-view-header-menu > div.wrap-button.row > div:nth-child(1) > a")
search_box.click()
driver.implicitly_wait(5)
search_box = driver.find_element('css selector', "#email")
search_box.send_keys("kimpansu666@gmail.com")
time.sleep(2)
search_box = driver.find_element('css selector', "#password")
search_box.send_keys("**********")
time.sleep(2)
search_box = driver.find_element('css selector', "#login-form > div.form-group.btn-submit.mt-4 > button")
search_box.click()
driver.implicitly_wait(5)

#loopでスクレイピング 
for j,y in zip(h,l):
    
    search_box=driver.find_element('css selector', '#home_search_text')
    driver.implicitly_wait(5)
    search_box.send_keys(j)
    time.sleep(2)
    search_box.send_keys(Keys.RETURN)
    driver.implicitly_wait(5)
    search_box=driver.find_element('css selector',"body > div.main-content-cls-fix-5.main-content.py-lg-5.py-md-5.py-0 > div.main-content-cls-fix-6.container.search-company-wrapper.hy-container > div > div.col-lg-9.col-md-8.col-12.wrap-content > div.result_form > div:nth-child(1) > div.d-flex.header-item.company_header > div > div > div.company-header__name > h2 > a")
    driver.implicitly_wait(5)
    search_box.click()
    driver.implicitly_wait(5)
    search_box=driver.find_element('css selector',"body > div.main-content-user > div.container.hy-container > div > div.col-lg-9-hyo.col-md-12.px-0.user-content > div > div.widget.widget-detail-company.pb-md-2.parent-top-banner-3-mobile > div.widget-content.bg-white.pd-30.pb-md-15px.pt-15px.pad-category > div > div.col-md-6.mb-md-3.pb-inherit.px-lg-2.px-md-2.px-0 > a:nth-child(3) > span.d-inline-block.ml-2.font-weight-bold.fs-1")
    driver.implicitly_wait(5)
    search_box.click()
    driver.implicitly_wait(5)


    list_stat = []
    list_star = []
    list_review = []
    list_date = []
    list_name = []
    driver.implicitly_wait(3)

    z = driver.current_url
    a1 = driver.find_element('class name',"number")
    a2 = int(a1.text)
    a3 = int(a2/35)+1
    print(a3)

    for i in range(a3):
        search_url = z + str("?pagenum=")+str(i+1)
        driver.get(search_url)
        total=driver.find_elements('class name',"widget.widget-comment")
        driver.implicitly_wait(3)
    
        for i in range(35):
        
            driver.implicitly_wait(3)
            try:
                a=total[i].find_element('class name',"row.category").text
                driver.implicitly_wait(3)
            except NoSuchElementException: 
                a=total[i].find_element('class name',"row.font-size-08").text 
                driver.implicitly_wait(3)
            except IndexError:
                break

            try:
                b=total[i].find_element('class name',"point.font-weight-bold.d-inline").text
                driver.implicitly_wait(3)
            except NoSuchElementException: 
                b=np.nan

            c=total[i].find_element('class name',"w-100").text
            driver.implicitly_wait(3)
            e=total[i].find_element('class name',"text-center.gray-border.border-radius-0.mw-100.float-right.pl-2.pr-2.d-lg-flex.d-md-flex.d-none.bg-white").text
            driver.implicitly_wait(3)

            list_stat.append(a)
            list_star.append(b)
            list_review.append(c)
            list_date.append(e)
            list_name.append(y)

            data = {"name" : list_name, "stat" :list_stat, "star":list_star,"review":list_review,"date":list_date }
            df = pd.DataFrame(data)
            df.to_csv("/r_analysis/0.newdata/"+str(y)+".csv", encoding = "utf-8-sig")
    print('スクレイピングしました！: {}'.format(j))
    driver.get(url)
        

print("スクレイピング終わりました。chrome driverを閉じます。")
driver.close()
